<a href="https://colab.research.google.com/github/sagar9926/Natural-Language-Processing/blob/main/IMDB_Review_sentiment_Logistic_regression_and_Naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing all required packages

import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"sagaragrawal","key":"66052d202b185535c38dd21805de1a5d"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 ! kaggle datasets list

ref                                                          title                                           size  lastUpdated          downloadCount  
-----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  
babyoda/women-entrepreneurship-and-labor-force               Women Entrepreneurship and Labor Force           1KB  2020-11-21 08:38:51            652  
szymonjanowski/internet-articles-data-with-users-engagement  Internet news data with readers engagement       3MB  2020-11-21 17:09:57            248  
sakshigoyal7/credit-card-customers                           Credit Card customers                          379KB  2020-11-19 07:38:44           1123  
imoore/2020-us-general-election-turnout-rates                2020 US General Election Turnout rates           4KB  2020-11-26 00:21:15            391  
alexgude/california-traffic-collision-data-from-switrs       California Traffic Collisio

In [ ]:
! kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
!cp /content/imdb-dataset-of-50k-movie-reviews.zip /content/data

In [ ]:
cd data

/content/data


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: IMDB Dataset.csv        


In [ ]:
!rm imdb-dataset-of-50k-movie-reviews.zip

In [ ]:
# read the training data
df  = pd.read_csv("./IMDB Dataset.csv")

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
# map positive to 1 and negetive to 0

df.sentiment = df.sentiment.apply(lambda x : 1 if x == "positive" else 0)

In [ ]:
# we create a new column calles kfold and fill it eith -1
df['kfold'] = -1

In [ ]:
# the next step is to randomize the rows of the data

# to shuffle your dataframe in-place and reset the index, you could do e.g.

df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# fetch labels

y = df.sentiment.values

In [ ]:
# initiate the kfold class from model_selection module

kf  = model_selection.StratifiedKFold(n_splits=5)

__StratifiedKFold__ is a variation of KFold. First, StratifiedKFold shuffles your data, after that splits the data into n_splits parts and Done. Now, it will use each part as a test set. Note that it only and always shuffles data one time before splitting.

With shuffle = True, the data is shuffled by your random_state. Otherwise, the data is shuffled by np.random (as default). For example, with n_splits = 4, and your data has 3 classes (label) for y (dependent variable). 4 test sets cover all the data without any overlap.

In [ ]:
kf

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [ ]:
for f,(t_,v_) in enumerate(kf.split(X = df, y = y )):
  df.loc[v_,'kfold'] = f

In [ ]:
df.kfold.value_counts()

4    10000
3    10000
2    10000
1    10000
0    10000
Name: kfold, dtype: int64

## How does CountVectorizer works?

https://www.youtube.com/watch?v=RZYjsw6P4nI

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# To create a Count Vectorizer, we simply need to instantiate one.
# There are special parameters we can set here when making the vectorizer, but
# for the most basic example, it is not needed.
vectorizer = CountVectorizer()

# For our text, we are going to take some text from our previous blog post
# about count vectorization
sample_text = ["One of the most basic ways we can numerically represent words "
               "is through the one-hot encoding method (also sometimes called "
               "count vectorizing)."]

# To actually create the vectorizer, we simply need to call fit on the text
# data that we wish to fix
vectorizer.fit(sample_text)

# Now, we can inspect how our vectorizer vectorized the text
# This will print out a list of words used, and their index in the vectors
print('Vocabulary: ')
print(vectorizer.vocabulary_)


Vocabulary: 
{'one': 12, 'of': 11, 'the': 15, 'most': 9, 'basic': 1, 'ways': 18, 'we': 19, 'can': 3, 'numerically': 10, 'represent': 13, 'words': 20, 'is': 7, 'through': 16, 'hot': 6, 'encoding': 5, 'method': 8, 'also': 0, 'sometimes': 14, 'called': 2, 'count': 4, 'vectorizing': 17}


In [ ]:
# If we would like to actually create a vector, we can do so by passing the
# text into the vectorizer to get back counts
vector = vectorizer.transform(sample_text)
print(vector)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	2
  (0, 13)	1
  (0, 14)	1
  (0, 15)	2
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1


In [ ]:
# Our final vector:
print('Full vector: ')
print(vector.toarray())

# Or if we wanted to get the vector for one word:
print('Hot vector: ')
print(vectorizer.transform(['hot']).toarray())

# Or if we wanted to get multiple vectors at once to build matrices
print('Hot and one: ')
print(vectorizer.transform(['hot', 'one']).toarray())


Full vector: 
[[1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1]]
Hot vector: 
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Hot and one: 
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]


In [ ]:
vectorizer.vocabulary_

{'also': 0,
 'basic': 1,
 'called': 2,
 'can': 3,
 'count': 4,
 'encoding': 5,
 'hot': 6,
 'is': 7,
 'method': 8,
 'most': 9,
 'numerically': 10,
 'of': 11,
 'one': 12,
 'represent': 13,
 'sometimes': 14,
 'the': 15,
 'through': 16,
 'vectorizing': 17,
 'ways': 18,
 'we': 19,
 'words': 20}

In [ ]:
vectorizer.vocabulary_

{'also': 0,
 'basic': 1,
 'called': 2,
 'can': 3,
 'count': 4,
 'encoding': 5,
 'hot': 6,
 'is': 7,
 'method': 8,
 'most': 9,
 'numerically': 10,
 'of': 11,
 'one': 12,
 'represent': 13,
 'sometimes': 14,
 'the': 15,
 'through': 16,
 'vectorizing': 17,
 'ways': 18,
 'we': 19,
 'words': 20}

In [ ]:
# We could also do the whole thing at once with the fit_transform method:
print('One swoop:')
new_text = ['Today is the one one that I do the thing today, today today today today today today ']
new_vectorizer = CountVectorizer()
print(vectorizer.transform(new_text))

One swoop:
  (0, 7)	1
  (0, 12)	2
  (0, 15)	2


## Logistic Regression

In [ ]:
# we go over the folds created

for fold_ in range(5):
  # temporary dataframes for train and test
  train_df = df[df.kfold != fold_].reset_index(drop = True)
  test_df = df[df.kfold == fold_].reset_index(drop = True)

  # initialize Countvectorizer with NLTK's word_tokenize
  # function as tokenizer

  count_vec = CountVectorizer(
      tokenizer = word_tokenize,
      token_pattern = None
  )

  # fit count_vec on training data reviews
  count_vec.fit(train_df.review)

  # transform training and validation data reviews
  xtrain = count_vec.transform(train_df.review)
  xtest = count_vec.transform(test_df.review)

  # initialize logistic regression model
  model = linear_model.LogisticRegression()

  # fit the model on training reviews and sentiment
  model.fit(xtrain,train_df.sentiment)

  # make predictions on test data
  # thershold for predictions is 0.5
  preds = model.predict(xtest)

  # calculate accuracy
  accuracy = metrics.accuracy_score(test_df.sentiment,preds)

  print(f'Fold : {fold_}')
  print(f"Accuracy = {accuracy}")
  print("")




/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold : 0
Accuracy = 0.8913



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold : 1
Accuracy = 0.8848



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold : 2
Accuracy = 0.8942



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold : 3
Accuracy = 0.8939

Fold : 4
Accuracy = 0.8968



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
xtrain

<40000x149180 sparse matrix of type '<class 'numpy.int64'>'
	with 5887920 stored elements in Compressed Sparse Row format>

In [ ]:
xtest

<10000x149180 sparse matrix of type '<class 'numpy.int64'>'
	with 1449096 stored elements in Compressed Sparse Row format>

In [ ]:
len(xtrain[0].toarray()[0])

149180

In [ ]:
xtrain[0].toarray()[0].max()

26

In [ ]:
count_vec.inverse_transform(xtrain[0])

[array(['$', "'", "'s", '(', ')', ',', '.', '/', '10,000', '2', '5', '<',
        '>', '`', 'a', 'about', 'accepts.', 'ago', 'alex', 'all', 'also',
        'an', 'and', 'anyone', 'are', 'as', 'ask', 'asks', 'at', 'attempt',
        'audience', 'aussie', 'australian', 'australians', 'average',
        'bad', 'bank', 'beach', 'being', 'black', 'blackrock',
        'boardgames', 'boss', 'boy', 'br', 'brother', 'brown', 'bryan',
        'but', 'by', 'byrne', 'carrying', 'chopper', 'churning', 'classic',
        'comedy', 'concept', 'connected', 'couple', 'course', 'crime',
        'crocodile', 'cross', 'day', 'dead', 'deepened', 'deliver', 'dept',
        'do', 'drag', 'earlier.', 'early', 'end', 'enjoyed', 'enter',
        'evolution', 'evolved', 'exciting', 'expect', 'favour', 'film',
        'find', 'for', 'found', 'from', 'gang', 'general', 'generally',
        'get', 'getting', 'good', 'gun', 'guy', 'hands', 'hands.', 'has',
        'he', 'head', 'heath', 'helping', 'here', 'himself',

In [ ]:
## Correctly classify negetive review

model.predict(count_vec.transform(["The movie was bad, I did not like it"]))

array([0])

In [ ]:
# failed to classify positive review
model.predict(count_vec.transform(["The movie was not that bad, I actually like it"]))

array([0])

## Naive Bayes

sentiment classification : https://www.youtube.com/watch?v=Zt83JnjD8zg

multinomial NB : https://www.youtube.com/watch?v=O2L2Uv9pdDA

In [ ]:

# we go over the folds created
from sklearn import naive_bayes
for fold_ in range(5):
  # temporary dataframes for train and test
  train_df = df[df.kfold != fold_].reset_index(drop = True)
  test_df = df[df.kfold == fold_].reset_index(drop = True)

  # initialize Countvectorizer with NLTK's word_tokenize
  # function as tokenizer

  count_vec = CountVectorizer(
      tokenizer = word_tokenize,
      token_pattern = None
  )

  # fit count_vec on training data reviews
  count_vec.fit(train_df.review)

  # transform training and validation data reviews
  xtrain = count_vec.transform(train_df.review)
  xtest = count_vec.transform(test_df.review)

  # initialize logistic regression model
  model = naive_bayes.MultinomialNB()

  # fit the model on training reviews and sentiment
  model.fit(xtrain,train_df.sentiment)

  # make predictions on test data
  # thershold for predictions is 0.5
  preds = model.predict(xtest)

  # calculate accuracy
  accuracy = metrics.accuracy_score(test_df.sentiment,preds)

  print(f'Fold : {fold_}')
  print(f"Accuracy = {accuracy}")
  print("")




Fold : 0
Accuracy = 0.8437

Fold : 1
Accuracy = 0.842

Fold : 2
Accuracy = 0.8414

Fold : 3
Accuracy = 0.8478

Fold : 4
Accuracy = 0.8459



In [ ]:
## Correctly classify negetive review

model.predict(count_vec.transform(["The movie was bad, I did not like it"]))

array([0])

In [ ]:
# failed to classify positive review
model.predict(count_vec.transform(["The movie was not that bad, I actually like it"]))

array([0])

#### Logistic regression performs better than naive bayes but takes more time